In [8]:
import copy


In [2]:
class nbp_m:
    
    # __init__ function checks if the inputted list/ matrix (list of lists) is of:
    # correct type
    # correct structure

    def __init__(self, matrix, rows = None):
        
        #list of types for input
        types = [type(element) for element in matrix]
        
        # matrix has to be a list
        if type(matrix) != list:
            raise ValueError('The matrix has to be a list:\n1 - list of lists.\n2 - list of elements with rows stated.')
            
        # if matrix given as list
        elif sum([t == list for t in types]) == 0:
            
            # only numerical values
            for e in matrix:
                if (type(e) != int and type(e) != float):
                    raise ValueError('The given list contains values that are not numerical.')
            
            # rows have to be stated
            if rows == None:
                raise ValueError('The number of rows have to be stated.')
                
            # number of rows has to be an integer bigger than 1
            elif (type(rows) != int) or rows < 2:
                raise ValueError('The number of rows has to be a numerical value that is bigger then 1.')
                
            # at least 4 elements to be considered a matrix
            elif len(matrix) < 4:
                raise ValueError('The matrix has to contain at least 4 elements.')
                
            # number of elements has to be a multiple of number of rows
            elif len(matrix) % rows != 0 :
                raise ValueError('''The number of elements in the given list has to be a multiple of the number of rows.\n
                Number of elements - {}\n
                Number of rows - {}'''.format(len(matrix), rows))
            
            # if nothing from above - then we can store the instance as matrix
            else:
                num = len(matrix)//rows
                m = [matrix[x:x+num] for x in range(0, len(matrix), num)]
                
                self.matrix = m
                self.rows = rows
                self.columns = len(m[0])

                    
        elif sum([t == list for t in types]) in  range(1,len(matrix)):
            raise ValueError('Both lists and non-lists in given list.')
        
        # if matrix given as a matrix - that is list of lists
        else:
            
            # if the matrix is given as list - number of rows should not be specified
            if rows != None:
                raise ValueError('Both rows and matrix as list of lists were given.')
            
            # The lists in matrix have to contain only numerical values
            for row in matrix:
                if not all([(type(e)==int or type(e)==float) for e in row]):
                    raise ValueError('Not all elements in the matrix given are numerical.')
            
            # rows have to be of the same length
            if not all([len(row) == len(matrix[0]) for row in matrix]):
                raise ValueError('Not all rows in the matrix given are of the same length.')
                
            # if nothing above - we can store the input
            self.matrix = matrix
            self.rows = len(matrix)
            self.columns = len(matrix[0])

            
    
    def __add__(self, other):
        if not (isinstance(other, nbp_m)):
            raise ValueError("Other must be of instance matrix (nbp_m).")
        
        if (len(self.matrix)!= len(other.matrix)) or (len(self.matrix[0]) != len(other.matrix[0])):
            raise ValueError('Matrices are not of the same size.')        
        m1 = self.matrix
        m2 = other.matrix
        out = []
        for row in range(len(m1)):
            out.append([m1[row][i] + m2[row][i] for i in range(len(m1[0]))])
        return nbp_m(out)
    
    def __sub__(self, other):
        if not (isinstance(other, nbp_m)):
            raise ValueError("Other must be of instance matrix (nbp_m).")        
 
        if (len(self.matrix)!= len(other.matrix)) or (len(self.matrix[0]) != len(other.matrix[0])):
            raise ValueError('Matrices are not of the same size.')
        m1 = self.matrix
        m2 = other.matrix
        out = []
        for row in range(len(m1)):
            out.append([m1[row][i] - m2[row][i] for i in range(len(m1[0]))])
        return nbp_m(out)


    
    ###################################################
    
    ### helping functions returned as lists and not matrices###

    
    def identity_matrix(dim):
        base = nbp_m([0 for i in range(dim*dim)], dim).matrix
        for i in range(dim):
            base[i][i] = 1
            
        return nbp_m(base)
    
    
    def minor(matrix, r, c):
        # taking a matrix and returning Minor M_{r,c}

        row_index = [i for i in range(len(matrix))]
        col_index = [i for i in range(len(matrix[0]))]
        row_index.remove(r-1)
        col_index.remove(c-1)

        minor = [matrix[row] for row in row_index]
        for row in range(len(minor)):
            minor[row] = [minor[row][col] for col in col_index]
        
        return nbp_m(minor)
    
    
    def is_square(self):
        return len(self.matrix) == len(self.matrix[0])
        
    def is_symmetric(self):
        return self.matrix == self.transpose().matrix

    
    ###################################################
    
    ### printing functions ###
    
    # function 'nice' is based on the Latex package
    
    def __str__(self):
        text = ''
        for row in self.matrix:
            text += str(row) + '\n'
        return '{}x{} matrix\n\n'.format(self.rows, self.columns) + text

            
    def nice(self):
        
        #The import statement is defined in the function due to reusability purposes
        #Furthermore, nice is a function purley for the user to have a nice display, and is therefore optional.
        #Having the import inside the function avoids importing a package that might not be used
        from IPython.display import display, Latex
        
        matrix_line = []
        r = self.rows
        c = self.columns

        # noting matrix in one line
        for row in self.matrix:
            matrix_line += row
        matrix_line = [round(num, 3) for num in matrix_line] # rounding so that the outcome is of normal size

        # coding the applicable string
        # using {} to control the formatting
        # using latex language and latex module
        text = ''
        text += '&'.join(['{}' for i in range(c)])

        for i in range(r-1):
            text += ' \\\ ' + '&'.join(['{}' for i in range(c)])

        gen = '$\\begin{{bmatrix}} '+text+' \end{{bmatrix}}$'
        
        gen_f = gen.format(*matrix_line)

        # printing
        display(Latex(f'{gen_f}'))
        
    ###################################################
    
    # functions for operating on matrices
    
    ### transpose function ###
    
    def transpose(self):
        
        c = self.columns
        r = self.rows
        t_m = [0 for i in range(self.rows*self.columns)]
        t_m = [t_m[x:x+r] for x in range(0, len(t_m), r)]
        
        for col in range(c):
            for row in range(r):
                t_m[col][row] = self.matrix[row][col]

        return nbp_m(t_m)

    ### scal function ###
    
    def scal(self, scalar):
        if (type(scalar) != int and type(scalar) != float):
            raise ValueError('The scalar parsed has to be a numerical value.')
        
        out = []
        for row in self.matrix:
            out.append([i*scalar for i in row])
            
        return nbp_m(out)
    
    ### Matrix multiplication and sum_matrix ###
    def MMULT(self, other):
        #Input validation
        if not (isinstance(other, nbp_m)):
            raise ValueError('other in MMULT must be an instance of the class nbp_m.')

        if not(self.columns == other.rows):
            raise ValueError('The number of columns in the first matrix must be equal to the number of rows in the second.')
            
        res = [[0 for x in range(self.columns)] for y in range(self.rows)] 
        for i in range(self.rows):
            for j in range(other.columns):
                for k in range(other.rows):
  
                    # resulted matrix
                    res[i][j] += self.matrix[i][k] * other.matrix[k][j]
                
        return nbp_m(res)
    
    #Adds the elements of the matrices together, returns it as a new matrix
    def sum_matrix(self, lst):
        
        for other in lst:
            if not (isinstance(other, nbp_m)):
                raise ValueError("other must be a matrix (instance of nbp_m).")
            if (self.rows != other.rows or self.columns != other.columns):
                raise ValueError("The matrices are of different dimensions.")
        
        out = []
        #takes the current row and adds it together with all other using the vector equivalent of this class
        for i in range(len(self.matrix)):
            out.append(nbp_v(self.matrix[i]).sum_vec([nbp_v(other.matrix[i]) for other in lst]).vector)

        return nbp_m(out)

      
 
     
    ###################################################
    
    # reducing rows, find pivots, row echelon forms
    
    # class-functions/ helping functions
    
    def pivot_row(row, row_count):
        value_count = 0
        skipped = 0
        for value in row:
                if(value_count == row_count):
                    if(value != 0):
                        return value, value_count+skipped, skipped
                    else:
                        skipped += 1
                        value_count-=1
                value_count+=1
        return "Null"

    def find_pivot(matrix):

        #pivtos{row_count: [value, (position)]}
        pivots = {}
        row_count = 0
        for row in matrix:
            if(nbp_m.pivot_row(row, row_count) == "Null"):
                row_count +=1
                continue
            pivots[row_count] = [nbp_m.pivot_row(row, row_count)[0], (row_count, nbp_m.pivot_row(row, row_count)[1])]
            for i in range(nbp_m.pivot_row(row, row_count)[2]):
                row_count +=1
            row_count +=1

        return pivots
    
    def switching_rows(reduced):
        ## changes the order of the matrix if row switching is necessary, 
        ## if it is not necessary, then it leaves the rows unchanged  
        row_count = 0
        for row in reduced:
            if(nbp_m.pivot_row(row,row_count) == "Null"):
                continue
            if(nbp_m.pivot_row(row,row_count)[2] > 0):
                for r in range(row_count+1, len(reduced)):
                    if(nbp_m.pivot_row(reduced[r],0) == "Null"):
                        row_count +=1
                        continue
                    if(nbp_m.pivot_row(reduced[r],0)[1] <= nbp_m.pivot_row(row,row_count)[1]):
                        a = copy.deepcopy(reduced[row_count])
                        reduced[row_count] = reduced[r]
                        reduced[r] = a
            row_count += 1
        return reduced
    
    
    #instance functions
        
    def row_echelon_form(self):
        #row-switches if necessary
        reduced = nbp_m.switching_rows(copy.deepcopy(self.matrix))
        row_count = 0
        row = reduced[row_count]
        while(row_count < len(reduced)-1):
            if(nbp_m.pivot_row(row, row_count) == "Null"):
                row_count += 1
                continue
            pivot_value = nbp_m.pivot_row(row, row_count)[0]
            pivot_pos = nbp_m.pivot_row(row, row_count)[1]
            for i in range(1,len(reduced)):
                if(row_count+i >= len(reduced)):
                    continue
                value_under = reduced[row_count+i][pivot_pos]
                factor = (-value_under)/pivot_value
                e_count = 0
                for e in reduced[row_count+i]:
                    reduced[row_count+i][e_count] += row[e_count]*factor
                    e_count +=1
            row_count += 1
            row = reduced[row_count]
        return nbp_m(reduced)


    def rre(self):
        #row-switches if necessary
        reduced = nbp_m.switching_rows(copy.deepcopy(self.matrix))     
        row_count = 0
        row = reduced[row_count]
        while(row_count < len(reduced)-1):
            if(nbp_m.pivot_row(row, row_count) == "Null"):
                row_count += 1
                continue
            pivot_value = nbp_m.pivot_row(row, row_count)[0]
            pivot_pos = nbp_m.pivot_row(row, row_count)[1]
            factor = (1/pivot_value)
            e_count = 0
            for e in reduced[row_count]:
                reduced[row_count][e_count] *= factor
                e_count += 1
            for i in range(1,len(reduced)):
                if(row_count+i >= len(reduced)):
                    continue
                value_under = reduced[row_count+i][pivot_pos]
                factor2 = (-value_under)
                e_count = 0
                for e in reduced[row_count+i]:
                    reduced[row_count+i][e_count] += row[e_count]*factor2
                    e_count +=1
            row_count += 1
            row = reduced[row_count]
        return nbp_m(reduced)

    
    
        #Based on rank = number of pivot columns in a matrix's row reduced echelon form 
    def rank(self):
        return len(nbp_m.find_pivot(self.rre().matrix).keys())
    
    
    def determinant(self):
            
        if not(self.is_square()):
            raise ValueError("The matrix must be square to compute the determinant.")
        if (self.rows != len(nbp_m.find_pivot(self.row_echelon_form().matrix).keys())):
            return 0
        
        determinant_lst = []
        for i in nbp_m.find_pivot(self.row_echelon_form().matrix).values():
            determinant_lst.append(i[0])
        determinant = 1
        for j in determinant_lst:
            determinant *= j
        return determinant
     
        
    def inverse(self):
        if self.determinant() == 0:
            raise ValueError('The matrix is singular (determinant = 0).')        
        
        #only used for indexing -> does not need to be a copy
        matrix = self.matrix
        determinant = self.determinant()
        

        if len(matrix) == 2:
            return nbp_m([matrix[1][1], -1*matrix[0][1], -1*matrix[1][0], matrix[0][0]],2).scal(1/determinant)

        inverse = nbp_m([0 for i in range(len(matrix)*len(matrix))],len(matrix)).matrix
        
        for i in range(len(matrix)):
            for j in range(len(matrix[0])):
                minor = nbp_m.minor(matrix, i+1, j+1)
                inverse[i][j] = ((-1)**(i+1+j+1)) * minor.determinant()

        return nbp_m(inverse).transpose().scal(1/determinant)



            
    def lu(self):
        if(self.determinant() == 0):
            raise ValueError("This LU-decomposition can only be done on invertable matrices, i.e. where the determinant is non-zero")
        
        if(self.matrix != nbp_m.switching_rows(copy.deepcopy(self.matrix))):
            raise ValueError("This LU-decomposition can only be performed on matrices where row operations is not required before decomposing.")
            
        a = copy.deepcopy(self.matrix)
        dim = len(a)
        l = nbp_m([0 for i in range(dim*dim)], dim).matrix
        u = nbp_m([0 for i in range(dim*dim)], dim).matrix
        
    
        for n in range(dim):
            l[n][0] = a[n][0]
            u[0][n] = a[0][n]/a[0][0]
            u[n][n] = 1
        
        for d in range(1, dim):
            for i in range(1, dim):
                if i >= d:
                    l[i][d] = a[i][d]
                    for k in range(d):
                        l[i][d] -= l[i][k]*u[k][d]
                else:
                    l[i][d] = 0
                
                if d <= i:
                    u[d][i] = a[d][i]
                    for k in range(d):
                        u[d][i] -= l[d][k]*u[k][i]
                    u[d][i] = u[d][i]/l[d][d]
                else:
                    u[d][i] = 0
    
        return {'l':nbp_m(l), 'u':nbp_m(u)}
    


    def solve_linear(self, vec):
        if not (isinstance(vec, nbp_v)):
            raise ValueError("The input has to be a vector (of the class nbp_v).")
            
        # Solving the linear equation: Ax = b 
        # Using LU decomposition: LUx = b
        # The first step: Ly = b
        # The second step: Ux = y
        
        L = self.lu()['l'].matrix
        U = self.lu()['u'].matrix
        b = vec.vector
    
        # The first step: using forward substitution to solve Ly = b for y
        y = [0 for i in range(len(b))]
        y[0] = b[0]/L[0][0]
        for n in range(1, len(b)):
            h = b[n]
            for j in range(n):
                h -= L[n][j]*y[j]
            y[n] = h/L[n][n]


        # The second step: using backward substitution to solve Ux = y for x
        x = [0 for i in range(len(b))]
        x[-1] = y[-1]/U[-1][-1]
        for n in range(len(b)-2, -1, -1):
            h = y[n]
            for j in range(n+1, len(b)):
                h -= U[n][j]*x[j]
            x[n] = h/U[n][n]

        
        return nbp_v(x,"v")
    
    
    def gram_schmidt(self):
    
        # The matrix must be of full rank, the columns must be lineary independent.
        if (self.rank() < self.columns):
            raise ValueError("The matrix must be of full rank to compute qr using this gram_scmidt")
    
        
        #e and u are stored as lists of vectors,
        #colA is stored as a matrix,
        #r is a list of lists.
        u = []
        e = []
        colA = self.transpose()
        
        u.append(nbp_v(colA.matrix[0]))
        
        e.append(u[0].div_vec(u[0].norm))
    
        r = []
        for i in range(len(colA.matrix)):
            r.append([])
        r[0].append(nbp_v(colA.matrix[0]).dot_product(e[0]))
        r[0] = r[0] + [0]*(len(colA.matrix)-len(r[0]))
        
    
        for i in range(1, len(colA.matrix)):

            #c is a vector (nbp_v)
            c = nbp_v(colA.matrix[i])
            u_new = copy.deepcopy(c)
        
            for e_vector in e:
                u_new = u_new.sum_vec([e_vector.scalar(e_vector.dot_product(c)).scalar(-1)])

            u.append(u_new)
            e_new = u_new.div_vec(u_new.norm)
            e.append(e_new)

            for e_vector in e:
                r[i].append(c.dot_product(e_vector))
                
            r[i] = r[i] + [0]*(len(colA.matrix)-len(r[i]))
    
        q = []
        for e_vector in e:
            q.append(e_vector.vector)
                
        return {'q':nbp_m(q).transpose(), 'r':nbp_m(r).transpose()}
    
    
    def householder(self):

        # AThe matrix must be of full rank, the columns must be lineary independent.
        if (self.rank() < self.columns):
            raise ValueError("The matrix must be of full rank to compute qr using this householder")

    
        # The algorithm is from Kincaid.
    
        #dim is stored as an int,
        #q, r and A are stored as a matrix (nbp_m).
        dim = self.rows
        q = nbp_m.identity_matrix(dim)
        r = copy.deepcopy(self)
        A = self.transpose()

        for k in range(dim-1):
        
            # a1 = first column of A
            a1 = nbp_v(A.matrix[0])

            # beta 
            beta = -1*(a1.vector[0]/abs(a1.vector[0]))*a1.norm

            # e1 = the unit vector of size equal to a1
            e1 = nbp_v([1] + [0 for i in range(len(A.matrix)-1)])

            # y = a1 - beta*e1
            beta_e1 = e1.scalar(-1*beta)
            y = a1.sum_vec([beta_e1])
        
            # alpha
            alpha = (2**0.5)/y.norm
        
            # v = alpha * y
            v = y.scalar(alpha)
        
            # v*v^T matrix
            vvT = v.outer_product(nbp_v(v.vector,"v"))
            
            # U = I - vvT
            U = nbp_m.identity_matrix(a1.dimensions).sum_matrix([vvT.scal(-1)])


            # In the first iteration U does not have to be adapted to the original matrix size,
            # but the next ones have to.
            # The missing upper right part is the according part of identity matrix
            if k > 0:
                for t in range(k):
                    U.matrix.insert(0, [0 for i in range(dim)])
                for row in range(dim):
                    if len(U.matrix[row]) < dim:
                        U.matrix[row] = [0]*(dim-len(U.matrix[row])) + U.matrix[row]
                for t in range(k):
                    U.matrix[t][t] = 1

            # Updating U with the new matrix
            U = nbp_m(U.matrix)
    
            # By repetetive multiplication (U*r) we should end up with upper triangular matrix r
            r = U.MMULT(r)

            # We obtain matrix Q by multiplying obtained U_k matrices and then transposing the final product
            q = U.MMULT(q)
            
            # Taking minor of A obtained after this step
            A = nbp_m.minor(r.matrix, 1, 1)
   
    
        return {'q':q.transpose(), 'r':r}
    
    
    
    def qr(self, method = "h"):
        if(method != "h" and method != "g"):
            raise ValueError("QR decomposition can be acchieved with the Gram Schmidt- or Householder methods, please select method = g or method = h.")
    
        if method == 'g':
            return {'q': self.gram_schmidt()["q"], 'r': self.gram_schmidt()["r"]}
        
        return {'q': self.householder()["q"], 'r': self.householder()["r"]}
    
    
    
    def qr_alg(self, iterations=100, method = "h"):
        
        ## We are also validating method here, so that no variables will be stored before qr is called. 
        if(method != "h" and method != "g"):
            raise ValueError("QR decomposition can be acchieved with the Gram Schmidt- or Householder methods, please select method = g or method = h.")
            
        if (self.rank() < self.columns):
            raise ValueError("The matrix must be of full rank to compute qr using this algorithm.")
        
        values = copy.deepcopy(self)
        vectors = copy.deepcopy(self)
  
        for i in range(iterations):

            # Storing values.qr(method) in v to not compute twice in order to improve performance (significantly when iterations are 100+) 
            v = values.qr(method)
            q_val = v['q']
            r_val = v['r']
            q_vec = vectors.qr(method)['q']
            
            values = r_val.MMULT(q_val)
            vectors = self.MMULT(q_vec)
        
        return {'m':values, 'eig':vectors.qr()['q']}

    
    
    def eigen(self, method = "h"):
        if not self.is_square():
            raise ValueError("Can only compute eigenvalues of a square matrix")
        
        if not self.is_symmetric():
            raise ValueError("this function can only compute eigenvalues for symmetric matrices")
    
        eigen_values = [round(self.qr_alg()['m'].matrix[i][i],3) for i in range(self.rows)]
        eigen_vectors = self.qr_alg()['eig'].transpose().matrix
        for k in range(len(eigen_vectors)):
            eigen_vectors[k] = [round(x,3) for x in eigen_vectors[k]]
    
        return {'values':nbp_v(eigen_values), 'vectors':nbp_m(eigen_vectors)}
    
    
    def diagonalize(self, method = "h"):
        if not (self.is_symmetric()):
            raise ValueError("Diagonalize can only be done for symmetric matrices, since that is a requirement for computing eigenvalues in this library")
        
        # The output is PDP^-1
        # D = matrix with eigenvalues
        # P = matrix containing of corresponding eigenvalues
        # P-1 = the transposed of matrix P
    
        D = nbp_m.identity_matrix(self.rows)
    
        for i in range(self.rows):
            D.matrix[i][i] = self.eigen(method)["values"].vector[i]
        

        
        P = self.eigen(method)["vectors"].transpose()

    
        return {'D':D, 'P':P, 'P-1':P.inverse()}





In [1]:
class nbp_v:

    # checks if inputted list/matrix (list of lists) is of:
    # correct type
    # correct structure
    
    # variables stored:
    # vector is a list of the vector elements
    # direction is for the user to specify if the vector is horizontal or vertical (h or v)
    def __init__(self, vector, direction = "h"):
        
        if type(vector) != list:
            raise ValueError('The vector-input has to be list')
           
        for e in vector:
            if not (type(e) == int or type(e) == float):
                raise ValueError("All the vector elements must be of type int or float")  
        
        self.dimensions = len(vector)
        self.vector = vector
        self.direction = direction
        self.norm = sum([i**2 for i in self.vector]) ** 0.5
        
        
    def __add__(self, other):
        if not (isinstance(other, nbp_v)):
            raise ValueError("other must be of instance vector (nbp_v).")        
        
        if (len(self.vector)!= len(other.vector)):
            raise ValueError('The vectors are not of the same dimension.')        
        v1 = self.vector
        v2 = other.vector
        out = []
        for i in range(len(v1)):
            out.append(v1[i] + v2[i])
        return nbp_v(out)
    
    def __sub__(self, other):
        if not (isinstance(other, nbp_v)):
            raise ValueError("other must be of instance vector (nbp_v).")        
   
        if (len(self.vector)!= len(other.vector)):
            raise ValueError('vectors are not of the same dimension.')        
        v1 = self.vector
        v2 = other.vector
        out = []
        for i in range(len(v1)):
            out.append(v1[i] - v2[i])
        return nbp_v(out)  


######## printing functions ######
    
    # Printing the vector as a list.
    def __str__(self):
        return str(self.vector)    
    

    def nice(self):
        
        #The import statement is defined in the function due to reusability purposes
        #Furthermore, nice is a function purley for the user to have a nice display, and is therefore optional.
        #Having the import inside the function avoids importing a package that might not be used
        
        from IPython.display import display, Latex
        

        vector_line = [round(num, 3) for num in self.vector] # rounding so that the outcome is of normal size

        # Coding the applicable string,
        # using {} to control the formatting,
        # and using latex to display the vector
        text = ''
        if(self.direction == "h"):
            text += '&'.join(['{}' for i in range(len(self.vector))])

        else:
            for i in range(len(self.vector)):
                text += '{}\\\\'
                
        gen = '$\\begin{{bmatrix}} '+text+' \end{{bmatrix}}$'
        gen_f = gen.format(*vector_line) #matrix_line


        # printing
        display(Latex(f'{gen_f}'))

 

    ### checking if two vectors are orthogonal ###

    def orthogonal(self, other):
        return self.dot_product(other) == 0
       


#### manipulating vectors #######        

        
    def scalar(self, scalar):
        if (type(scalar) != int and type(scalar) != float):
            raise ValueError('The scalar parsed must be numerical.')        
        return nbp_v([i*scalar for i in self.vector], self.direction)
    
    def div_vec(self, divisor):
        if (type(divisor) != int and type(divisor) != float):
            raise ValueError('The divisor parsed must be numerical.') 
        return nbp_v([i/divisor for i in self.vector], self.direction)
    
    def sum_vec(self, lst):
        for vec in lst:
            if not (isinstance(vec, nbp_v)):
                raise ValueError("All of the other input have to be vectors (instance of nbp_v).")
        
        dimensions = [i.dimensions for i in lst]
        dimensions.append(self.dimensions)
        lst.append(self)
        if [dimensions[0]]*len(dimensions) != dimensions:
            raise ValueError('Some of the vectors have different dimensions.')
            
        out = []
        for i in range(dimensions[0]):
            out.append(sum([k.vector[i] for k in lst]))
                       
        return nbp_v(out)
    
    def linear_independent(self, other):
        if(self.direction!= other.direction or self.dimensions != other.dimensions):
            return True
        lst =[]
        if(self.direction == "h"):
            lst.append(self.vector)
            lst.append(other.vector)
        else:
            for i in range(len(self.vector)):
                lst2 = []
                lst2.append(self.vector[i])
                lst2.append(other.vector[i])
                lst.append(lst2)
        return nbp_m(lst).rank() >1
    
    def dot_product(self, other):
        if not (isinstance(other, nbp_v)):
            raise ValueError("other has to be of instance nbp_v.")
        if (self.dimensions != other.dimensions):
            raise ValueError('Multiplication cannot be done due to different dimensions.')
        return sum([self.vector[i]*other.vector[i] for i in range(len(self.vector))])
    
    def cross_product(self, other):
        if not (isinstance(other, nbp_v)):
            raise ValueError("other has to be of instance nbp_v.")
        if (self.dimensions != other.dimensions):
            raise ValueError('Cross product cannot be done due to different dimensions.')
        if (self.dimensions != 3):
            raise ValueError("Cross product is only defined for 3 and 7 dimensions, this function can only compute for 3D.")
        a = self.vector
        b = other.vector
        return nbp_v([a[1]*b[2] - a[2]*b[1],
                      a[2]*b[0] - a[0]*b[2],
                      a[0]*b[1] - a[1]*b[0]],self.direction) 
    
    def outer_product(self, other):
        if not (isinstance(other, nbp_v)):
            raise ValueError("other has to be of instance nbp_v")
        if (self.dimensions != other.dimensions):
            raise ValueError('Outer product cannot be done due to different dimensions')
            
        # The outer product of the first vector (m x n), and the second vector (n x j) result in the matrix: m x j
    
        d1 = len(self.vector)
        d2 = len(other.vector)
    
        # The function returns an instance of nbp_m.
        res = nbp_m([0 for i in range(d1*d2)], d1)
    
        for i in range(d1):
            for j in range(d2):
                res.matrix[i][j] = self.vector[i]*other.vector[j]
                
        return res
    
    

         

In [6]:
d = nbp_v([1,1,1],"v")
e = nbp_v([2,2,3])
f = nbp_v([2,2,3,3])
g = nbp_v([5,4,2,3])
h = nbp_v([8,-10])


#nbp_m([[1,2],[1,2],[1,2]]).nice()

#d+e
#d-e
#d.nice()
#e.nice()
#d.dot_product(e)
#e.scalar(3).nice()
#e.div_vec(3).nice()
#print(d.linear_independent(e))
#e.length
#d.VMULT(e).nice()
#e.VMULT(d).nice()
#d.VMULT(f).nice()
#f.VMULT(d).nice()
#lst = []
#lst.append(e)
#lst.append(f)
#lst.append([1,2,3])
#d.sum_vec(lst).nice()

#g.orthogonal(h)
#g.dot_product(h)

#a = nbp_m([1,2,3,
#           4,5,6,
#           7,8,8],
#          3)
#a.is_symmetric()
#a.diagonalize()

#a.inverse().nice()

#we = nbp_m([-0.523,0.813,-0.258,-0.554,-0.553,-0.622,-0.648,-0.183,0.739],3)

#f.cross_product(g).nice()

#print(d+e)

#print(d-e+e)

#d.outer_product(e).nice()

print("f")
f.nice()
print("e")
e.nice()

f.outer_product(e).nice()
e.outer_product(f).nice()




f


<IPython.core.display.Latex object>

e


<IPython.core.display.Latex object>

ValueError: Outer product cannot be done due to different dimensions

In [242]:
a = nbp_m([1,1,3,
           0,0,6,
           2,0,0],
          3)

g = nbp_m([1,1,1,2],2)
#a.row_echelon_form().nice()
#a.rre().nice()


a.nice()

#a.lu()["l"].nice()
a.lu()["u"].nice()

<IPython.core.display.Latex object>

ValueError: This LU-decomposition can only be performed on matrices where row operations is not required before decomposing.

In [275]:
a = nbp_m([2, 6, 9, 6, 8, 5, 9, 5, 8], 3)
d = nbp_m([1,2,3,
           4,5,6,
           7,8,8],
          3)
#a.row_echelon_form().nice()
#a.determinant()
#a.rank()
#a.householder()["q"].nice()
#a.householder()["r"].nice()
#b = nbp_m([1,2,
 #          3,4],
  #        2)

#b.row_echelon_form().nice()
#b.rre().nice()
#print("rank(a): "+str(a.rank()))
#print("rank(b): "+str(b.rank()))
#print("det(a): "+str(a.determinant()))
#print("det(b): "+str(b.determinant()))

#a.inverse().nice()
#b.inverse().nice()

#a.MMULT(d).nice()

print("other")
a.nice()
print("self")
d.nice()

d.MMULT(a).nice()

a.rre().nice()
d.rre().nice()


other


<IPython.core.display.Latex object>

self


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

rank(a): 2
rank(b): 3
det(a): 0
det(b): -1.0


In [245]:

a = nbp_m([1,2,3,
           4,5,6,
           7,8,8],
          3)
#print(a.determinant() == 0)
b = nbp_m([1,1,1,2,2,2,3,3,3],3)
c = nbp_v([1,1,1],"v")
d = nbp_m([1,1,1,2,2,2,3,3,3],3)
f = nbp_m([1,2,3,
           4,5,6,
           7,8,8,9,10,11],
          4)
g = nbp_m([1,1,1,1,1,1,1,1,1],3)
h = nbp_m([2, 6, 9, 6, 8, 5, 9, 5, 8], 3)

#print(a)
#a.nice()
#a.rre().nice()
#print("l")
#a.lu()["l"].nice()
#print("u")
#a.lu()["u"].nice()

print("input: ")
h.nice()
#print(h.is_symmetric() == True)
#print(h.is_symmetric())


#c.nice()
#h.solve_linear(c)
#b.rre().nice()
#b.rank()
#f.nice()


#a.sum_matrix([b,d]).nice()

#a.MMULT(f).nice()
#f.MMULT(a).nice()
#b.MMULT(b).nice()

#q = a.householder()["q"]
#q.MMULT(a.householder()["r"]).nice()
#a.nice()

#q2 = a.qr()['q']
#q2.MMULT(a.qr()['r']).nice()

#a.MMULT(nbp_m.identity_matrix(a.rows)).nice()
#a.MMULT(q2).nice()
#q2.nice()
#a.qr()["r"].MMULT(q2).nice()

#a.qr_alg()['m'].nice()
#a.qr_alg()['eig'].nice()
#a.is_square()
#a.is_symmetric()
#g.transpose().nice()
#g.is_symmetric()
import time

start = time.time()
s = h.diagonalize()
s["P"].nice()
s["D"].nice()
s["P-1"].nice()
end = time.time()
print("qr using householder time: "+str(end-start))


start2 = time.time()
s = h.diagonalize(method = "g")
s["P"].nice()
s["D"].nice()
s["P-1"].nice()
end2 = time.time()
print("qr using g-s time: "+str(end2-start2))

#h.nice()



#s["P"].nice()
#s["D"].nice()
#s["P-1"].nice()
#s["P"].inverse().nice()

#ss = s["P"].MMULT(s["D"])

#ss.MMULT(s["P-1"]).nice()

#nbp_m.matrix([1,2,3,4],1)

#print("h")
#h.nice()
#print("l*u")
#h.lu()["l"].MMULT(h.lu()["u"]).nice()







input: 


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

qr using householder time: 4.567142724990845


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

qr using g-s time: 4.917922258377075


In [1104]:
b = nbp_m([1,1,1,2,2,2,3,3,3],3)
print(b.rank())
print(b.columns)
b.dim_null()

c = nbp_v([0,0,1],"v")
c.norm

1
3


1.0

In [892]:

q = nbp_m([9,2,3,
           4,9,6,
           7,8,9],
          3)
a = nbp_m([9,2,3,
           4,9,6,
           7,8,9],
          3)
a.rows
a.columns
a.matrix
print(q.MMULT(a))
q.scal(3).nice()
q.determinant()
q.inverse().nice()
#a.is_square()
#q.determinant()
#nbp_m.row_echelon_form(q)
#nbp_m.identity_matrix(4).nice()
#nbp_m.zeros(3).nice()
a+q
a-q
#q.MMULT([1,2,3])


3x3 matrix

[110, 60, 66]
[114, 137, 120]
[158, 158, 150]



<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

[[0, 0, 0]]

In [52]:
b = nbp_m([5,3,8,5,8,8,6,2,2,1,2,3],3)
a-b
#c = nbp_m([[1,2,3],[2,3,4],["pedro","odd","jack"]])

[[-4, -1, -5, -1]]

In [33]:
d = nbp_m([[1,2,3],"PEDRO"])
"""
    def operation(self, other, op):
        
        if (len(self.matrix)!= len(other.matrix)) or (len(self.matrix[0]) != len(other.matrix[0])):
            raise ValueError('Matrices are not of the same size!')        
        m1 = self.matrix
        m2 = other.matrix
        out = []
        if(op == "+"):
            for row in range(len(m1)):
                out.append([m1[row][i] + m2[row][i] for i in range(len(m1[0]))])
        else:
            for row in range(len(m1)):
                out.append([m1[row][i] - m2[row][i] for i in range(len(m1[0]))])            
                
        return out
            
    def __add__(self, other):
        return operation(self, other, "+")
    
    def __sub__(self, other):
        return operation(self, other, "-")
"""


ValueError: Both lists and non-lists in given list!

In [40]:
a.nice()

<IPython.core.display.Latex object>

In [41]:
#a.nice()
a.transpose()
a.find_pivot()


{0: [1, (0, 0)], 1: [6, (1, 1)], 2: [11, (2, 2)]}

In [8]:
b.nice()

<IPython.core.display.Latex object>

In [14]:
c.nice()

<IPython.core.display.Latex object>

In [ ]:
    ### Matrix multiplication ###
    def MMULT(self,other):           
    res = [[0 for x in range(len(matrix1))] for y in range(len(matrix1[0]))] 
  
    # explicit for loops
    for i in range(len(matrix1)):
        for j in range(len(matrix2[0])):
            for k in range(len(matrix2)):
  
                # resulted matrix
                res[i][j] += matrix1[i][k] * matrix2[k][j]
                
    return res